# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных
   
Для анализа представлен файл data.csv, содержащий следующую информацию:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита 

## Откройте таблицу и изучите общую информацию о данных

Открываем файл c данными и сохраняем его в переменной data. На экран выводим первые десять строк таблицы, а так же оцениваем объем данных:

In [1]:
import pandas as pd
import numpy as np

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [3]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод** Файл содержит 12 столбцов и 21525 сторк. 

В 2 столбцах тип данных - `float`, в 5 - `int`, `object` - в 5.


Для анализа представлены следующие колонки:
- `children` — для проверки зависимости между наличием детей и возвратом кредита в срок;
- `family_status` — для проверки зависимости между семейным положением и возвратом кредита в срок;
- `family_status_id` — для проверки зависимости между семейным положением и возвратом кредита в срок;
- `total_income` — для проверки зависимости между уровнем дохода и возвратом кредита в срок;
- `purpose` — для проверки как разные цели кредита влияют на его возврат в срок.
- Колонка `debt` содержит сведения о наличии задолженности по возврату кредитов.

## Предобработка данных

### Обработка пропусков

In [5]:
data.isna().sum().sort_values(ascending=False)[data.isna().sum().sort_values(ascending=False) > 0]

days_employed    2174
total_income     2174
dtype: int64

Поскольку пропущенные данные носят количественный характер, заполним их характерными значениями с помощью метода median(). Данный метод лучше подходит для расчета среднего дохода, если имеются выбросы.

При этом, будем использовать группировку уровня дохода в зависимости от типа занятости (income type).

Проверим, действительно ли зависит средний доход от типа занятости:

In [6]:
data.groupby('income_type')['total_income'].median().sort_values(ascending=False)

income_type
предприниматель    499163.144947
компаньон          172357.950966
госслужащий        150447.935283
сотрудник          142594.396847
безработный        131339.751676
пенсионер          118514.486412
студент             98201.625314
в декрете           53829.130729
Name: total_income, dtype: float64

Исправим пропуски в среднем доходе:

In [7]:
data['total_income'] = data['total_income'].fillna(data.groupby(['income_type'])['total_income'].transform('median'))

Так же заполним пропуски в `days_employed`:

In [8]:
data['days_employed'] = data['days_employed'].fillna(data.groupby(['income_type'])['days_employed'].transform('median'))

### Замена типа данных

Колонка `total_income` содержит вещественный формат данных, поменяем на `int`:

In [9]:
data['total_income'] = data['total_income'].astype('int')

### Обработка аномальных значений

В данных содержаться анимальные значения. Заменим отрицательный стаж:

In [10]:
data['days_employed'] = data['days_employed'].abs()

Проверим количество детей в семье `children`:

In [11]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В датасете найдены аномальные значение - отрицательное количество детей и `20` детей. Исправим это:

In [12]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Проверим исправления:

In [13]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Убедимся, что предобработка данных проведена корректно:

In [14]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Обработка дубликатов

Обработаем явные и неявные дубликаты в `education`:

In [15]:
data['education'] = data['education'].str.lower()

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

Произведем категоризацию данных на основании диапазонов, указанных ниже:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Посмотрим на перечень уникальных целей для кредита в колонке `purpose`:

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Так же катетеризуем данные следующим образом:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [23]:
(
    data
    .pivot_table('debt', index='children',aggfunc=['count','sum','mean'])
    .droplevel(1,axis=1)
    .rename(columns={'count':'total','sum':'debt','mean':'relative_amount_debt_div_by_total'})
    .style
    .format({'relative_amount_debt_div_by_total': '{:,.2%}'.format})
    .highlight_max(color='red', subset='relative_amount_debt_div_by_total')
    .highlight_min(color='green',subset='relative_amount_debt_div_by_total')
    .set_caption('debt/children')
)

,total,debt,relative_amount_debt_div_by_total
children,,,
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%


**Вывод:** 

Из таблицы `result_debt_vs_amount_of_children` можно сделать следующие выводы: 

1. Самый маленький процент просроченных кредитов у семей без детей. Процент составляет `7.54%`;

2. В семьях с пятью и более детьми отсутсвуют просрочки по кредитам. Однако это объясняется не объективными факторами, а малой выборкой, ведь общее количество таких семей составляет всего $9$ домохозяйств;

3. Что касается остальных групп, то при  увеличении числа детей в семье наблюдается плавный рост примерно на `0.2%`. Исключение составляют семьи, в которых три ребенка. Здесь процент просрочек равен `8.18%`. 

Таким образом, количество просроченных кредитов у семей без детей и с детьми четко различается. При этом при увеличении количества детей в семье наблюдается плавный рост количества просрочек. Однако бывают статистические аномалии, как в случае группы семей с тремя детьми. В связи с чем, не представляется возможным сделать однозначный вывод о том, что между количеством детей в семье (где есть хотя бы один ребёнок) и просрочками по кредиту существует корреляция.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [24]:
(
    data
    .pivot_table('debt', index='family_status',aggfunc=['count','sum','mean'])
    .droplevel(1,axis=1)
    .rename(columns={'count':'total','sum':'debt','mean':'relative_amount_debt_div_by_total'})
    .style
    .format({'relative_amount_debt_div_by_total': '{:,.2%}'.format})
    .highlight_max(color='red', subset='relative_amount_debt_div_by_total')
    .highlight_min(color='green',subset='relative_amount_debt_div_by_total')
    .set_caption('debt/family_status')
)

,total,debt,relative_amount_debt_div_by_total
family_status,,,
Не женат / не замужем,2796,273,9.76%
в разводе,1189,84,7.06%
вдовец / вдова,951,63,6.62%
гражданский брак,4134,385,9.31%
женат / замужем,12261,927,7.56%


**Вывод:** 

Из сводной таблицы `debt/family_status` можно сделать следующие выводы:
1. Средний процент просрочек среди всех групп составляет - `8.06%`. При этом выборку нельзя назвать сбалансированной по количеству наблюдений;

2. Наибольшая группа по числу кредитов - `женат/замужем`. Относительный процент просрочек здесь ниже среднего и составляет `7.56%`, из чего можно сделать аккуратный вывод, что пары, состоящие в браке, явлются надежными заемщиками;

3. Самый маленький процент просрочек у группы `вдовец/вдова`  -  `6.62%`. Однако стоит учитывать количество наблюдений, здесь выборка (по естественным причинам) наименьшая из всех групп и составляет всего `951` человека, что практически в 13 раз меньше чем у группы из пункта 2. Таким образом, данные могут оказаться не репрезентативными в отдельности от других факторов;

4. То же самое, что и в пункте 3, можно сказать про группу `в разводе`, где процент просрочек составаляет `7.06%`, при выборке в `1189` наблюдений;

4. Самый большой процент у группы `Не женат/не замужем` - `9.76%`. При этом количество наблюдений в выборке соостветует медианному значению. Так, категория `не женат/не замужем` является наименее надежным заемщиком.

5. Следом идёт группа `гражданский брак` - `9.31%`. Количество домохозяйств: `4134`, что позволяет также данную категорию назвать ненадежными заемщиками.

Таким образом, таблицу `debt/family_status` можно поделить на две категории
- Количество наблюдений `~1000`. Не смотря на маленький средний процент - `6.84%` просрочек у таких групп как `в разводе` и `вдовец/вдова`, нельзя сделать однознаный вывод о их надежности;
- Количество наблюдений равное медианному и больше. Здесь наименьший процент просрочек у наибольшей группы `женат/замужем` в связи с чем это наиболее благонадежные заёмщики. У групп `не женат/не замужем` и `гражданский брак` же процент просрочек наибольший среди всех групп, в связи с чем их можно назвать наименее благонадежными заёмщиками.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [25]:
(
    data
    .pivot_table('debt', index='total_income_category',aggfunc=['count','sum','mean'])
    .droplevel(1,axis=1)
    .rename(columns={'count':'total','sum':'debt','mean':'relative_amount_debt_div_by_total'})
    .style
    .format({'relative_amount_debt_div_by_total': '{:,.2%}'.format})
    .highlight_max(color='red', subset='relative_amount_debt_div_by_total')
    .highlight_min(color='green',subset='relative_amount_debt_div_by_total')
    .set_caption('debt/total_income_category')
)

,total,debt,relative_amount_debt_div_by_total
total_income_category,,,
A,25,2,8.00%
B,5014,354,7.06%
C,15921,1353,8.50%
D,349,21,6.02%
E,22,2,9.09%


**Вывод:** 

Зависимости между уровнем дохода и возратом кредита в срок нет. Можно предположить, что причина кроется в неизвестных переменных, например, таких как разные цели кредита.

Самое большое количество задолженностей - `9.09%` у группы `E` с самым маленьким уровнем доходов. При этом, следом идет группа `D` с наименьшим процентом просроченных кредитов - `6.02%`. 

Однако стоит учесть количество наблюдений в каждой выборке. Так, группы `E`, `D`, `A` представлены малым количеством домохозяйств, в свзяи с чем выводы могут быть искажены. 

Что касается хорошо представленных групп, таких как `B` (`7.06%`)  и `C` (`8.5%`), то здесь с ростом дохода падает и количество просрочек. Однако, в связи с маленьким количеством групп, однознаного вывода о том, что рост дохода и возврат кредита в срок это связанные велечины, сделать нельзя. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [26]:
(
    data
    .pivot_table('debt', index='purpose_category',aggfunc=['count','sum','mean'])
    .droplevel(1,axis=1)
    .rename(columns={'count':'total','sum':'debt','mean':'relative_amount_debt_div_by_total'})
    .style
    .format({'relative_amount_debt_div_by_total': '{:,.2%}'.format})
    .highlight_max(color='red', subset='relative_amount_debt_div_by_total')
    .highlight_min(color='green',subset='relative_amount_debt_div_by_total')
)

,total,debt,relative_amount_debt_div_by_total
purpose_category,,,
операции с автомобилем,4279,400,9.35%
операции с недвижимостью,10751,780,7.26%
получение образования,3988,369,9.25%
проведение свадьбы,2313,183,7.91%


**Вывод:** 

Наименьший процент просрочек наблюдается у группы с целью проведения операций с недвижимостью  - `7.26%`. Вероятно это связано с различными факторами, такими как: 

- различные льготные программы;

- первоночальный взнос; 

- возможность получение дохода со сдачи в аренду;

и др.факторами.

Наибольший же процент просрочек можно наблюдать у групп "операции с автомобилем" - `9.35%` и "получение образования" - `9.25%`. 

Так, можно сделать однозначный вывод, что различные цели кредита влияют на вероятность его просрочки.

Стоит отметить, что полный анализ стоит производить сразу по нескольким факторам. В качестве примера создадим сводную таблицу процентов задолженности по кредиту сразу по нескольким факторам.  

In [27]:
(
    data
    .loc[data.children != 5]
    .pivot_table('debt', index=['family_status', 'purpose_category', 'children']).unstack()
    .style.format("{:.2%}", na_rep='no_values').background_gradient(cmap = "YlOrRd")
)

Благодаря общей сводной таблице можно проанализировать каждый конкретный случай с помощью аппарата статистики. 

Однако не стоит забывать, что бывают искажения представления информации. Так, несмотря на то, что у категории `не женатых/не замужем` с целью кредита `операция с недвижимостью`, имеющих три и более ребенка в семье составляет $75$% (`50%` для трех детей и `100%` для четырех детей),  нельзя  сделать однозначный вывод о неблагонадежности данной группы. Это связанно с тем, что семей, попадающих под эти категории - $3$ домохозяйства.

В то же время, данные могут помочь при скоринге клиента. 

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* 

1. Технические ошибки:

2. Пропуски, связанные с человеческим фактором:

- Неправильно записанный или перезаписанный программно файл; 

- "Неважность" поля при его заполнение пользователем. Пример: email адрес;

- Отсутсвие какой-либо информации. Пример: у некоторых людей нет отчества;

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* 

Потому что зачастую количественные переменные не соответствуют Гауссовскому распределению, имея различные аномальные значения. В таком случае, среднее значение будет смещено в сторону аномалии и не будет репрезентативным, в то время как медианное значение будет показывать середину выборки.

### Шаг 4: общий вывод.

1. Наличие или отсутсвие детей, ожидаемо, влияет на платежеспобность клиента. Так, самый маленький процент просрочек у семей без детей и составляет - `7.54%`. Далее можно наблюдать плавный рост от группы к группе с большим количеством детей. Данный рост составляет примерно `~0.2%`. Исключение составляют домохозяйства с количеством детей пять и более, где средний процент просрочек составляет `0%`, однако это явление обосновано малым количеством наблюдений;

2. При этом нет четкой зависмости между уровнем дохода и количеством просроченных кредитов. Вывод нельзя назвать однозначным из-за несбалансированности групп по количеству наблюдений. Начиная с групп со средней зарплатой и выше мы видим снижение просрочек по кредиту с `8.5%` у группы `C` до `7.06%` у категории `B`. При этом у группы `A`, чей доход является наибольшим, средний процент просрочек составляет `8%`, что не позволяет сделать о тенденции к снижению просрочек с ростом уровня дохода. Стоит отметить, что наблюдений в данной категории - `25`, в связи с чем данные могут не отражать дейсвительность;

3. Что касается семейного статуса, то здесь четкое разделение между категориями. Наименьший относительный процент просрочек среди хорошо представленных категорий у группы `женат/замужем` и составляет `7.56%`, в связи с чем данную группу можно назвать благонадежными заемщиками. У групп `не женат/не замужем` и `гражданский брак` же процент просрочек наибольший среди всех групп (при достаточном количестве наблюдений), в связи с чем их можно назвать наименее благонадежными заёмщиками. Две оставшиеся группы: `вдовец/вдова` и `в разводе` имеют малый процент просрочек, однако в связи с тем, что выборка является малой, стоит оценивать дополнительные факторы;

4. Цель кредита также сильно влияет на вероятность его возврата в срок. Здесь можно выделить две категории:
- Наименьшее количество просрочек - `операции с недвижимостью` - `7.26` и `проведение свадьбы` - `7.91`;
- Наибольшее количество просрочек - `операции с автомобилем` - `9.35` и `получение образования` - `9.25`.

Для скоринга клиента стоит прибегать к оценке совокупности факторов, ведь только так оценка вероятности просрочки кредита окажется наиболее справедливой. 

Можно составить портерта клиента. Наиболее надежный заемщик:

* Не имеет детей

* вдовец/вдова или в разводе 

* Цель кредита - операции с недвижимостью 

Наименее надежный заемщик:

* Имеет 4 и более детей

* Не женат

* Цель кредита - операции с автомобилем